In [1]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: docstring-parser
    Found existing installation: docstring-parser 0.15
    Uninstalling docstring-parser-0.15:
      Successfully uninstalled docstring-parser-0.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.


In [2]:
import sys
import os

import numpy as np
import pandas as pd
import torch
import transformers
import datasets

from tqdm import tqdm
from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from trl import RewardTrainer, RewardConfig

2024-08-05 14:15:53.081593: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-05 14:15:53.081694: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-05 14:15:53.198492: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
PATH = "distilbert/distilbert-base-cased"

reward_model = AutoModelForSequenceClassification.from_pretrained(
    PATH, 
    num_labels=1
)
tokenizer = AutoTokenizer.from_pretrained(PATH)

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [4]:
dataset = load_dataset("stanfordnlp/imdb")

train_dataset = dataset["train"]

positive_comments = [tokenizer.encode_plus(comment["text"], 
                                           truncation=True, padding="max_length",
                                           max_length=128) for comment in train_dataset if comment["label"] == 1][0:1000]
negative_comments = [tokenizer.encode_plus(comment["text"],
                                           truncation=True, padding="max_length",
                                           max_length=128) for comment in train_dataset if comment["label"] == 0][0:1000]

pairs = [(pos, neg) for pos in positive_comments for neg in negative_comments]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
def preprocess_function(pairs):
    pair = {
        "input_ids_chosen": [],
        "attention_mask_chosen": [],
        "input_ids_rejected": [],
        "attention_mask_rejected": [],
    }
    for chosen, rejected in tqdm(pairs):

        pair["input_ids_chosen"].append(chosen["input_ids"])
        pair["attention_mask_chosen"].append(chosen["attention_mask"])
        pair["input_ids_rejected"].append(rejected["input_ids"])
        pair["attention_mask_rejected"].append(rejected["attention_mask"])

    return pair

processed_pairs = preprocess_function(pairs)

processed_dataset = Dataset.from_dict(processed_pairs)

100%|██████████| 1000000/1000000 [00:02<00:00, 384324.05it/s]


In [6]:
training_args = RewardConfig(
    output_dir=".",  
    save_strategy="no",     
    num_train_epochs=1,     
    report_to="none",      
    learning_rate=1e-6,
    per_device_train_batch_size=64,
)

trainer = RewardTrainer(
    model=reward_model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=processed_dataset,
)

/opt/conda/lib/python3.10/site-packages/trl/trainer/reward_trainer.py:175: UserWarning: When using RewardDataCollatorWithPadding, you should set `max_length` in RewardConfig. It will be set to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/reward_trainer.py:192: UserWarning: When using RewardDataCollatorWithPadding, you should set `remove_unused_columns=False` in your RewardConfig we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Проверил, можно на 2000 степах просто остановить и уже будет окей

In [7]:
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2778: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss


KeyboardInterrupt: 

In [8]:
reward_model.save_pretrained("reward_model")

# Проверочка

In [8]:
dataset = load_dataset("stanfordnlp/imdb")

train_dataset = dataset["test"]

positive_comments = [tokenizer.encode_plus(comment["text"], truncation=True, padding="max_length", max_length=128) for comment in train_dataset if comment["label"] == 1][0:100]
negative_comments = [tokenizer.encode_plus(comment["text"], truncation=True, padding="max_length", max_length=128) for comment in train_dataset if comment["label"] == 0][0:100]

pairs = [(pos, neg) for pos in positive_comments for neg in negative_comments]

from datasets import load_dataset, Dataset, DatasetDict

In [9]:
reward_model.to("cuda")


def preprocess_function(pairs):
    pair = {
        "input_ids_chosen": [],
        "attention_mask_chosen": [],
        "input_ids_rejected": [],
        "attention_mask_rejected": [],
    }
    for chosen, rejected in tqdm(pairs):

        pair["input_ids_chosen"].append(chosen["input_ids"])
        pair["attention_mask_chosen"].append(chosen["attention_mask"])
        pair["input_ids_rejected"].append(chosen["input_ids"])
        pair["attention_mask_rejected"].append(chosen["attention_mask"])

    return pair

processed_pairs = preprocess_function(pairs)

processed_dataset = Dataset.from_dict(processed_pairs)

targets = []
labels = []
for i in tqdm(range(len(train_dataset))):
    tokens = tokenizer.encode_plus(train_dataset[i]["text"], truncation=True, padding="max_length", max_length=128) 

    output = reward_model.forward(input_ids=torch.tensor(tokens["input_ids"]).view(1, -1).to("cuda"),
                     attention_mask=torch.tensor(tokens["attention_mask"]).view(1, -1).to("cuda"))["logits"].item()
    output = max(0, output)
    if output != 0:
        output = 1
        
    targets.append(output)
    labels.append(train_dataset[i]["label"])

from sklearn.metrics import accuracy_score
accuracy_score(labels, targets)

100%|██████████| 25000/25000 [03:20<00:00, 124.44it/s]


0.80912